All info here: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

Code here: https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

## Import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, Dropout, Attention, Bidirectional, Lambda, TimeDistributed, Concatenate, Dot
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLearning(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('categorical_accuracy'))
        self.val_acc.append(logs.get('val_categorical_accuracy'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="crossentropy")
        ax1.plot(self.x, self.val_losses, label="val_crossentropy")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="accuracy")
        ax2.plot(self.x, self.val_acc, label="val_accuracy")
        ax2.legend()
        
        plt.show();

In [ ]:
# from google.colab import files
# uploaded = files.upload()

## Preparation of the dataset

### Compute inputs (for encoder and decoder) and target

In [ ]:
def tokenizationForRhymeEncoding(word_to_idx, max_len_word):
  words_tokenized = dict()
  words_tokenized[0] = np.zeros(max_len_word, dtype=np.int32)
  #Encoding of the rhyme (OPTIONAL)
  char_to_idx = {"'": 17, '(': 26, ')': 24, 'a': 1,  'b': 19, 'c': 9, 'd': 11, 'e': 3, 'f': 18, 'g': 14, 'h': 20, 'i': 2, 'j': 25, 'l': 10, 'm': 12, 'n': 6, 'o': 4, 'p': 13, 'q': 22, 'r': 5, 's': 8, 't': 7, 'u': 15, 'v': 16, 'x': 23, 'y': 27, 'z': 21}
  for (word,idx) in word_to_idx.items():
    char_list = []
    for c in word:
      char_list.append(char_to_idx.get(c, 0))
    words_tokenized[idx] = pad_sequences([char_list], maxlen=16, value = 0)[0]
  return words_tokenized

In [ ]:
#Prepare dataset for training: input for encoder and decoder and target for decoder
CHARACTER_BASED = False
RHYME_ENCODING = False and (not CHARACTER_BASED)
LINE_SEPARATOR = '$'
encoder_input_texts = []
decoder_input_texts = []
decoder_target_texts = []
NUM_VERSES_PREVIOUS_CONTEXT = 3
max_encoder = 0
max_decoder = 0
word_max_len = 16

with open("/content/gdrive/My Drive/DC-poem-formatUTF_withTercet.txt", encoding='latin-1') as file:
  text = file.read()

START = "1" if CHARACTER_BASED else "<start>"
END = "2" if CHARACTER_BASED else "<end>"
TERCET = "3" if CHARACTER_BASED else "<tercet>"
BOL = "4" if CHARACTER_BASED else "<bol>" 
EOL = "5" if CHARACTER_BASED else "<eol>"

if CHARACTER_BASED:
  text = text.replace("<start>", START).replace("<end>", END).replace("<tercet>", TERCET).replace("<bol>", BOL).replace("<eol>", EOL)
else:
  text = text.replace(BOL, BOL + " ").replace(EOL, " " + EOL)
datasetTexts = text.split('\n')

tokenizer = Tokenizer(char_level=CHARACTER_BASED, filters='!"#%&()*+,-./:;=?@[\\]^_`{|}~\t\n', oov_token="<oov_token>")
tokenizer.fit_on_texts(datasetTexts + [LINE_SEPARATOR])
token_to_idx = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
tokenized_datasetTexts = tokenizer.texts_to_sequences(datasetTexts)

if RHYME_ENCODING:
  rhymeEncodingInputByWord = tokenizationForRhymeEncoding(token_to_idx, word_max_len)
  rhyme_model = load_model('/content/gdrive/My Drive/RhymeEncoding.h5', compile=False) 

for i in range(0, len(tokenized_datasetTexts)):
  #encoder inputs (compute previous context of a verse)
  if i < len(tokenized_datasetTexts)-1:
    encoder_sentence = []
    for j in range(NUM_VERSES_PREVIOUS_CONTEXT-1, -1, -1):
      if i-j>=0:
        if tokenized_datasetTexts[i-j] == tokenized_datasetTexts[0]:
          encoder_sentence = tokenized_datasetTexts[i-j][1:-1] + [token_to_idx[LINE_SEPARATOR]]
        else:
          encoder_sentence += tokenized_datasetTexts[i-j][1:-1] + [token_to_idx[LINE_SEPARATOR]]      
    encoder_input_texts.append(encoder_sentence)
    max_encoder = max(max_encoder, len(encoder_input_texts[i]))
  #decoder input and target
  if i>0:
    decoder_input_sentence = tokenized_datasetTexts[i][0:-1]
    decoder_input_texts.append(decoder_input_sentence)
    decoder_target_texts.append(tokenized_datasetTexts[i][1:])
    max_decoder = max(max_decoder, len(decoder_input_texts[i-1]))

### Definition of the dataset

In [ ]:
from tensorflow.keras.utils import Sequence
import math
import random

class Seq2seqDataset(Sequence):
    def __init__(self, encoder_input, decoder_input, decoder_target, max_encoder=None, max_decoder=None, batch_size=128, shuffle=False, tokenizer_for_rhyme=None):  
      self.X = [encoder_input, decoder_input]
      self.Y = decoder_target
      self.batch_size = batch_size
      self.indexes = list(range(len(self.Y)))
      self.shuffle_after_epoch = shuffle
      self.tokenizer_for_rhyme = tokenizer_for_rhyme

    def shuffle(self):
      random.shuffle(self.indexes)
      return self

    def on_epoch_end(self):
      if self.shuffle_after_epoch:
        self.shuffle()

    def __len__(self):
      return math.ceil(len(self.X[0])/self.batch_size)

    def __getitem__(self, idx):
      indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
      batch_encoder_input = pad_sequences([self.X[0][i] for i in indexes], maxlen=max_encoder, padding='pre', value=0, dtype=np.int32)
      batch_decoder_input = pad_sequences([self.X[1][i] for i in indexes], maxlen=max_decoder, padding='pre', value=0, dtype=np.int32)
      batch_decoder_target = pad_sequences([self.Y[i] for i in indexes], maxlen=max_decoder, padding='pre', value=0, dtype=np.int32)
      if self.tokenizer_for_rhyme == None:
        return [batch_encoder_input, batch_decoder_input], batch_decoder_target
      else:
        batch_encoder_rhyme_input = np.asarray([[self.tokenizer_for_rhyme[token] for token in text] for text in batch_encoder_input])
        batch_decoder_rhyme_input = np.asarray([[self.tokenizer_for_rhyme[token] for token in text] for text in batch_decoder_input])
        return [batch_encoder_input, batch_encoder_rhyme_input, batch_decoder_input, batch_decoder_rhyme_input], batch_decoder_target

In [ ]:
encoder_input_data = np.asarray(encoder_input_texts)
#encoder_input_rhyme_data = np.asarray(encoder_input_rhyme_texts)
decoder_input_data = np.asarray(decoder_input_texts)
#decoder_input_rhyme_data = np.asarray(decoder_input_rhyme_texts)
decoder_target_data = np.asarray(decoder_target_texts)
decoder_target_data = np.asarray([to_categorical(target, num_classes=vocab_size) for target in decoder_target_data])

batch_size = 128

validation = False 
valid_split = 0.1
#split_treshold = int(encoder_input_data.shape[0]*(1-valid_split))
#This treshold indicate where the first canto that will be present in validation set starts 
#(validation split is computed on the 100 canti instead of the total number of verses)
split_treshold = 17007

if validation:
  if RHYME_ENCODING:
    train_set = Seq2seqDataset(encoder_input_data[:split_treshold], 
                              decoder_input_data[:split_treshold], decoder_target_data[:split_treshold], 
                              max_encoder=max_encoder, max_decoder=max_decoder, batch_size=batch_size, tokenizer_for_rhyme=rhymeEncodingInputByWord)
    valid_set = Seq2seqDataset(encoder_input_data[split_treshold:], decoder_input_data[split_treshold:], decoder_target_data[split_treshold:], 
                              max_encoder=max_encoder, max_decoder=max_decoder, batch_size=batch_size, tokenizer_for_rhyme=rhymeEncodingInputByWord)
  else:
    train_set = Seq2seqDataset(encoder_input_data[:split_treshold], decoder_input_data[:split_treshold], decoder_target_data[:split_treshold], 
                              max_encoder=max_encoder, max_decoder=max_decoder, batch_size=batch_size)
    valid_set = Seq2seqDataset(encoder_input_data[split_treshold:], decoder_input_data[split_treshold:], decoder_target_data[split_treshold:], 
                              max_encoder=max_encoder, max_decoder=max_decoder, batch_size=batch_size)
else:
  if RHYME_ENCODING:
    train_set = Seq2seqDataset(encoder_input_data, 
                              decoder_input_data, decoder_target_data, 
                              max_encoder=max_encoder, max_decoder=max_decoder, batch_size=batch_size, tokenizer_for_rhyme=rhymeEncodingInputByWord)
  else:
    train_set = Seq2seqDataset(encoder_input_data, decoder_input_data, decoder_target_data, max_encoder=max_encoder, 
                              max_decoder=max_decoder, batch_size=batch_size)

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## Model definition

### Seq2Seq

In [ ]:
#param
latent_dim = 128
token_vector_space_dimension = 100

# Define an input sequence and process it.
encoder_inputs = Input(shape=(max_encoder), name="Encoder input")
embedding = Embedding(vocab_size, token_vector_space_dimension, mask_zero=True, name="Embedding")
encoder_embedding = embedding(encoder_inputs)

#encoder = LSTM(latent_dim, dropout=0.2, return_sequences=True, return_state=True)
#encoder_outputs, state_h, state_c = encoder(encoder_embedding)
encoder = Bidirectional(LSTM(latent_dim, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=True, name="Encoder"))

if RHYME_ENCODING:
  encoder_inputs_rhyme = Input(shape=(max_encoder, word_max_len), name="Encoder input for rhyme")
  time_distributed_rhyme_model = TimeDistributed(rhyme_model)
  encoder_input_rhyme = time_distributed_rhyme_model(encoder_inputs_rhyme)
  encoder_inputs_with_rhyme_encoding = Concatenate()([encoder_embedding, encoder_input_rhyme]) 
  encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs_with_rhyme_encoding)
else:
  encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_embedding)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(max_decoder), name="Decoder input")
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_embedding = embedding(decoder_inputs)
decoder = LSTM(latent_dim*2, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, return_state=True, name="Decoder")

if RHYME_ENCODING:
  decoder_inputs_rhyme = Input(shape=(max_decoder, word_max_len), name="Decoder input for rhyme")
  decoder_input_rhyme = time_distributed_rhyme_model(decoder_inputs_rhyme)
  decoder_inputs_with_rhyme_encoding = Concatenate()([decoder_embedding, decoder_input_rhyme])
  decoder_outputs, decoder_state_h, _ = decoder(decoder_inputs_with_rhyme_encoding, initial_state=encoder_states)
else:
  decoder_outputs, decoder_state_h, _ = decoder(decoder_embedding, initial_state=encoder_states)

# decoder1 = LSTM(latent_dim*2, return_sequences=True, return_state=True)
# decoder2 = LSTM(latent_dim*2, return_sequences=True, return_state=True)
# decoder_outputs, decoder_state_h, _ = decoder2(decoder1(decoder_embedding, initial_state=encoder_states))

context_vector = Attention(dropout=0.2)([decoder_outputs, encoder_outputs])
dense_input_layer = Concatenate()([decoder_outputs, context_vector])

decoder_dense = Dense(vocab_size, activation='softmax')
model_outputs = decoder_dense(dense_input_layer)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
if RHYME_ENCODING:
  model = Model([encoder_inputs, encoder_inputs_rhyme, decoder_inputs, decoder_inputs_rhyme], model_outputs)
else:
  model = Model([encoder_inputs, decoder_inputs], model_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder input (InputLayer)      [(None, 13)]         0                                            
__________________________________________________________________________________________________
Encoder input (InputLayer)      [(None, 33)]         0                                            
__________________________________________________________________________________________________
Embedding (Embedding)           multiple             1377900     Encoder input[0][0]              
                                                                 Decoder input[0][0]              
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 33, 256), (N 234496      Embedding[0][0]              

### Training

In [ ]:
model = loaded_model

In [ ]:
epoch_interval = 10
interval_batches_saving = len(train_set)*epoch_interval

model_filepath="/content/gdrive/My Drive/Dante_model/WordSeq2SeqBiAttention_Dropout_0.2-final-{epoch:02d}+50.hdf5"
# Define Callbacks
es = EarlyStopping(monitor='categorical_crossentropy', mode='min', verbose=2, patience=5)
ck = ModelCheckpoint(model_filepath, verbose=1, save_freq=interval_batches_saving)
# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(train_set, epochs=70, callbacks=[ck, es, PlotLearning()])

In [ ]:
from tensorflow.keras import backend as K
def perplexity(y_true, y_pred):
  return K.exp(K.mean(K.categorical_crossentropy(y_true, y_pred)))

### Inference

In [ ]:
attention = True
latent_dim = 128

def extract_inference_models(model, rhyme_encoding):
  if rhyme_encoding:
    #input of tokenized words
    encoder_input = Input(shape=(max_encoder))
    embedding = model.layers[4]
    encoder_embedding = embedding(encoder_input)
    #input for rhyme encoding
    encoder_rhyme_input = Input(shape=(max_encoder, word_max_len))
    encoder_rhyme_encoding = model.layers[5](encoder_rhyme_input)
    encoder_refined_input = Concatenate()([encoder_embedding, encoder_rhyme_encoding])
    #encoder
    encoder_lstm = model.layers[7]
    #encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_refined_input)
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    encoder_states = [state_h, state_c]
    if attention:
      encoder_model = Model([encoder_input, encoder_rhyme_input], [encoder_outputs, encoder_states])
    else:
      encoder_model = Model([encoder_input, encoder_rhyme_input], encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim*2))
    decoder_state_input_c = Input(shape=(latent_dim*2))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_input = Input(shape=(max_decoder))
    decoder_embedding = embedding(decoder_input)

    decoder_rhyme_input = Input(shape=(max_decoder, word_max_len))
    decoder_rhyme_encoding = model.layers[5](decoder_rhyme_input)
    decoder_refined_input = Concatenate()([decoder_embedding, decoder_rhyme_encoding])

    decoder_lstm1 = model.layers[11]
    dense_layer = model.layers[-1]
    decoder_outputs, state_h_inf, state_c_inf = decoder_lstm1(decoder_refined_input, initial_state=decoder_states_inputs)

    if attention:
      encoder_state_input_for_attention = Input(shape=(max_encoder, latent_dim*2))
      attention_inf = model.layers[12]
      attention_output_inf = attention_inf([decoder_outputs, encoder_state_input_for_attention])
      dense_input_layer_inf = Concatenate()([decoder_outputs, attention_output_inf])
      decoder_states = [state_h_inf, state_c_inf]
      model_outputs = dense_layer(dense_input_layer_inf)
      decoder_model = Model([decoder_input, decoder_rhyme_input] + decoder_states_inputs + [encoder_state_input_for_attention], [model_outputs] + decoder_states)
    else:
      decoder_states = [state_h_inf, state_c_inf]
      model_outputs = dense_layer(decoder_outputs)
      decoder_model = Model([decoder_input, decoder_rhyme_input] + decoder_states_inputs, [model_outputs] + decoder_states)
  #NOT Rhyme Encoding
  else:
    encoder_input = Input(shape=(max_encoder))
    embedding = model.layers[2]
    encoder_embedding = embedding(encoder_input)
    encoder_lstm = model.layers[3]
    #encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    encoder_states = [state_h, state_c]
    if attention:
      encoder_model = Model(encoder_input, [encoder_outputs, encoder_states])
    else:
      encoder_model = Model(encoder_input, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim*2))
    decoder_state_input_c = Input(shape=(latent_dim*2))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_input = Input(shape=(max_decoder))
    decoder_embedding = embedding(decoder_input)
    decoder_lstm1 = model.layers[6]
    dense_layer = model.layers[-1]
    decoder_outputs, state_h_inf, state_c_inf = decoder_lstm1(decoder_embedding, initial_state=decoder_states_inputs)

    if attention:
      encoder_state_input_for_attention = Input(shape=(max_encoder, latent_dim*2))
      attention_inf = model.layers[7]
      attention_output_inf = attention_inf([decoder_outputs, encoder_state_input_for_attention])
      dense_input_layer_inf = Concatenate()([decoder_outputs, attention_output_inf])
      decoder_states = [state_h_inf, state_c_inf]
      model_outputs = dense_layer(dense_input_layer_inf)
      decoder_model = Model([decoder_input] + decoder_states_inputs + [encoder_state_input_for_attention], [model_outputs] + decoder_states)
    else:
      decoder_states = [state_h_inf, state_c_inf]
      model_outputs = dense_layer(decoder_outputs)
      decoder_model = Model([decoder_input] + decoder_states_inputs, [model_outputs] + decoder_states)
  return encoder_model, decoder_model

In [ ]:
encoder_model, decoder_model = extract_inference_models(model, False)

In [ ]:
RHYME_ENCODING = False
loaded_model = load_model('/content/gdrive/My Drive/Dante_model/WordSeq2SeqBiAttention-Dropout_0.3-final-110.hdf5')
encoder_model, decoder_model = extract_inference_models(loaded_model, RHYME_ENCODING)

In [ ]:
#encoder_model.summary()
#decoder_model.summary()

In [ ]:
def compute_rhyme_input_from_tokenized_input(tokenized_input):
  return np.asarray([[rhymeEncodingInputByWord[token] for token in text] for text in tokenized_input])

from tensorflow import make_ndarray, make_tensor_proto
def decode_sequence(sequence, label = ""):
  return tokenizer.sequences_to_texts([sequence])[0]

def encode_sequence(text):
  return tokenizer.texts_to_sequences([text])[0]
  
def generate_sequence(encoder_model, decoder_model, input_seq):
    # Generate empty target sequence of length 1.
    if  CHARACTER_BASED:
      target_seq = tokenizer.texts_to_sequences("4")
    else:
      target_seq = tokenizer.texts_to_sequences(["<bol>"])
    
    target_seq = pad_sequences(target_seq, maxlen=max_decoder, padding='pre', value=0)
    input_seq = pad_sequences(input_seq, maxlen=max_encoder, padding='pre', value=0)

    # Encode the input as state vectors.
    if RHYME_ENCODING:
      encoder_input = [input_seq, compute_rhyme_input_from_tokenized_input(input_seq)]
    else:
      encoder_input = [input_seq]
    if attention:
      encoder_outputs, states_value = encoder_model(encoder_input)
    else:
      states_value = encoder_model(input_seq)

    attention_input = make_ndarray(make_tensor_proto(encoder_outputs))


    #print("Input:", input_seq.shape)
    #print("Start: ", target_seq)

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    i = 0
    while not stop_condition:
        if RHYME_ENCODING:
          decoder_input = [target_seq, compute_rhyme_input_from_tokenized_input(target_seq)]
        else:
          decoder_input = [target_seq]
        if attention:
          output_tokens, h, c = decoder_model(decoder_input + states_value + [encoder_outputs])
        else:
          output_tokens, h, c = decoder_model(decoder_input + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tokenizer.sequences_to_texts([[sampled_token_index]])[0]
        #print("Output token: " , sampled_token)

        if (sampled_token == EOL or len(decoded_sentence) == max_decoder):
            stop_condition = True
        else:
          decoded_sentence += sampled_token if CHARACTER_BASED else sampled_token + " "
          i+=1
          # Exit condition: either hit max length
          # or find stop character.
        
          # Update the target sequence (of length 1).
          #Pass only the last character generated to the decoder (suggested)
          target_seq = tokenizer.texts_to_sequences([sampled_token])
          #Pass all the character generated in the sequence to the decoder
          #target_seq = [target_seq.tolist()[0] + tokenizer.texts_to_sequences(sampled_token)[0]]
          target_seq = pad_sequences(target_seq, maxlen=max_decoder, truncating='pre', padding='pre', value=0)
          # Update states
          #print(h, " + ", c, " = ", h+context_vector)
          states_value = [h, c]

    #printSequenceFromEncodedArray(encoded_sentence, "Encoded (copying during generation)")
    #print(encoded_sentence.shape)
    return decoded_sentence

def generate_sequence_beam_search(encoder_model, decoder_model, input_seq, k=2):
    # Generate empty target sequence of length 1.
    if  CHARACTER_BASED:
      target_seq = tokenizer.texts_to_sequences(BOL)
    else:
      target_seq = tokenizer.texts_to_sequences([BOL])
    
    target_seq = pad_sequences(target_seq, maxlen=max_decoder, padding='pre', value=0)
    input_seq = pad_sequences(input_seq, maxlen=max_encoder, padding='pre', value=0)

    # Encode the input as state vectors.
    if RHYME_ENCODING:
      encoder_input = [input_seq, compute_rhyme_input_from_tokenized_input(input_seq)]
    else:
      encoder_input = [input_seq]
    if attention:
      encoder_outputs, states_value = encoder_model(encoder_input)
    else:
      states_value = encoder_model(input_seq)

    #attention_input = make_ndarray(make_tensor_proto(encoder_outputs))
    
    decoded_sentence = ''
    generated_output = dict()
    queue = [{"prob": 0.0,
          "decoded_sentence": "",
          "target_seq": target_seq,
          "states": states_value}]
    i = 0
    while len(queue) > 0:
      next_step_queue = []
      for context in queue:
        if RHYME_ENCODING:
          decoder_input = [context["target_seq"], compute_rhyme_input_from_tokenized_input(target_seq)]
        else:
          decoder_input = [context["target_seq"]]
        if attention:
          output_tokens, h, c = decoder_model(decoder_input + context["states"] + [encoder_outputs])
        else:
          output_tokens, h, c = decoder_model(decoder_input + context["states"])
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_tokens_sorted = np.argsort(output_tokens[0, -1, :])[-k:]

        decoded_sentence = context["decoded_sentence"]
        for sampled_token_index in sampled_tokens_sorted:
          sampled_token = tokenizer.sequences_to_texts([[sampled_token_index]])[0]
          if (sampled_token == EOL or i == max_decoder):
            generated_output[context["prob"]/(i+1)] = context["decoded_sentence"]
          else:
            decoded_sentence += sampled_token if CHARACTER_BASED else sampled_token + " "
          
            # Update the target sequence (of length 1).
            #Pass only the last character generated to the decoder (suggested)
            target_seq = tokenizer.texts_to_sequences([sampled_token])
            #Pass all the character generated in the sequence to the decoder
            #target_seq = [target_seq.tolist()[0] + tokenizer.texts_to_sequences(sampled_token)[0]]
            target_seq = pad_sequences(target_seq, maxlen=max_decoder, truncating='pre', padding='pre', value=0)
            # Update states
            #print(h, " + ", c, " = ", h+context_vector)
            states_value = [h, c]

            next_step_queue.append({"prob": np.log(output_tokens[0, -1, :][sampled_token_index])+context["prob"],
            "decoded_sentence": decoded_sentence,
            "target_seq": target_seq,
            "states": states_value})
      if len(next_step_queue)==0:
        queue = []
      else:
        queue = sorted(next_step_queue, key=lambda k: k['prob'])[:k]
        next_step_queue = []
      i+=1
    next_sentence_max_prob = max([key for key in generated_output.keys()])
    return generated_output[next_sentence_max_prob]

generated_text = []
generated_text.append(encoder_input_data[0])
output = []
print("start: ", tokenizer.sequences_to_texts([generated_text[0]])[0])
decoded_sentence= ""
while END not in decoded_sentence:
  context = []
  for i in range(0, len(generated_text)):
    context += generated_text[i]
  #print("Context passed: ", context)
  decoded_sentence = generate_sequence_beam_search(encoder_model, decoder_model, [context], k=1)
  if len(generated_text) == NUM_VERSES_PREVIOUS_CONTEXT:
    generated_text.pop(0)
  generated_text.append(encode_sequence(decoded_sentence + LINE_SEPARATOR))
  print(decoded_sentence)
  #print('Input sentence:', input_texts[seq_index])

start:  <start> $
o poca nostra nobiltà di sangue 
che glorïar di te la gente fai 
qua giù dove l'affetto nostro langue 
<tercet> 
mirabil cosa non mi sarà mai 
ché là dove appetito non si torce 
dico nel cielo a me punir di gloriai 
<tercet> 
se tu se' che se' di là raccorce 
da la sua donna parte di fuor di costoro 
ma che la natura tua sì risponda 
<tercet> 
poco sofferse poi che non mi dice 
non basta da la bella persona 
prima che faccia li occhi suoi e 'l ventre 
<tercet> 
così vid' io te così vid' io corse 
vidi un di lor volume sia lasso 
sì che 'l convento de la mia radice 
<tercet> 
voi mi tacea e quel padre sterpi 
sovra noi dinanzi a noi un più partito 
e tue vidi già ch'al mio disire 
<tercet> 
io son vòlto i passi e li occhi al piè io 
per lo stremo del foco e quando i pesi 
e io e vidi un fei mi fei 
<tercet> 
mentre che la carità che fu quinci 
carcere vai per altezza d'ingegno 
mi disse tu se' che tu se' ben certo 
<tercet> 
e io a lui tuo tuo veder li occhi 
che vïole